# Time Series Anomaly Detection Exercises
### Kwame V. Taylor


* Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? What are two hypotheses you can test? Use time series method for detecting anomalies, like exponential moving average with %b.

**Bonus:**
 * Can you label students who are viewing both the web dev and data science curriculum?
 * Can you label students by the program they are in?
 * Can you label users by student vs. staff?
 * What are Zach, Maggie, Faith, and Ryan's ids?

### Imports

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

### Acquire

In [2]:
colnames = ['date', 'timestamp', 'request_method', 'user_id', 'cohort_id', 'ip']

df = pd.read_csv('anonymized-curriculum-access.txt', header=None, index_col=False,
                 names=colnames, delim_whitespace=True, na_values='"-"')
#                 usecols=[0, 1, 2, 5])
df.head()

,date,timestamp,request_method,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61


### Prepare

In [3]:
# merge date and timestamp
df["ds"] = df["date"] +" "+ df["timestamp"]
df.head()

,date,timestamp,request_method,user_id,cohort_id,ip,ds
0,2018-01-26,09:55:03,/,1,8,97.105.19.61,2018-01-26 09:55:03
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61,2018-01-26 09:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,2018-01-26 09:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,2018-01-26 09:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2018-01-26 09:56:24


In [4]:
# drop date and timestamp
df = df.drop(columns=['date', 'timestamp'])
df.head()

,request_method,user_id,cohort_id,ip,ds
0,/,1,8,97.105.19.61,2018-01-26 09:55:03
1,java-ii,1,8,97.105.19.61,2018-01-26 09:56:02
2,java-ii/object-oriented-programming,1,8,97.105.19.61,2018-01-26 09:56:05
3,slides/object_oriented_programming,1,8,97.105.19.61,2018-01-26 09:56:06
4,javascript-i/conditionals,2,22,97.105.19.61,2018-01-26 09:56:24


In [5]:
# convert date column to datetime type
df.ds = pd.to_datetime(df.ds)
df.dtypes

request_method            object
user_id                    int64
cohort_id                 object
ip                        object
ds                datetime64[ns]
dtype: object

In [6]:
# set ds as index and sort
# this is a very important step!
df = df.set_index('ds').sort_index()
df.head()

,request_method,user_id,cohort_id,ip
ds,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61


In [7]:
df.shape

(719459, 4)

In [8]:
df.request_method.value_counts()

/                           40122
search/search_index.json    15393
javascript-i                14551
toc                         14018
java-iii                    10835
                            ...  
students/882/EDIT               1
7-clustering/dbscan             1
capstone/54                     1
fundamentals/views              1
appendix/extra-exercises        1
Name: request_method, Length: 2154, dtype: int64

In [9]:
df.ip.value_counts()

97.105.19.58       268648
97.105.19.61        60530
192.171.117.210      8896
71.150.217.33        4919
12.106.208.194       4262
                    ...  
107.77.219.231          1
173.127.33.133          1
172.56.6.146            1
187.237.231.72          1
70.123.229.111          1
Name: ip, Length: 4064, dtype: int64

In [10]:
df.cohort_id.nunique()

386

Some IP addresses accidentally got put in the ```cohort_id``` column, so I'm going to drop those rows.

In [11]:
df = df.drop(df[df.cohort_id.str.len() > 4].index)
df

,request_method,user_id,cohort_id,ip
ds,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61
...,...,...,...,...
2020-11-02 16:48:13,javascript-i/coffee-project,763,62,107.192.148.199
2020-11-02 16:48:17,javascript-i/mapbox-api,771,62,172.125.226.175
2020-11-02 16:48:18,javascript-i/coffee-project,771,62,172.125.226.175


In [12]:
#from ryan
#mask = (df.column.str.contains(“A”) | df.column.str.contains(“B”))
#df[mask]

In [13]:
df.isna().sum()

request_method      0
user_id             0
cohort_id         420
ip                  0
dtype: int64

In [14]:
df = df.dropna()

**Split IP address into network and host**
 * **Reference**: https://docs.oracle.com/cd/E19504-01/802-5753/planning3-18471/index.html

In [15]:
df[['network1','network2', 'host1', 'host2']] = df.ip.str.split(".",expand=True)
df.head()

,request_method,user_id,cohort_id,ip,network1,network2,host1,host2
ds,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,97,105,19,61
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,97,105,19,61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,97,105,19,61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,97,105,19,61
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,97,105,19,61


In [16]:
df['network'] = df['network1'] + df['network2']
df['host'] = df['host1'] + df['host2']
df.head()

,request_method,user_id,cohort_id,ip,network1,network2,host1,host2,network,host
ds,,,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,97,105,19,61,97105,1961
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,97,105,19,61,97105,1961
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,97,105,19,61,97105,1961
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,97,105,19,61,97105,1961
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,97,105,19,61,97105,1961


**I notice that there is at least one instance of a different user_id under the same exact IP address. Might be a typo, or just something I don't understand about IP addresses.**

In [17]:
df = df.drop(columns=['network1', 'network2', 'host1', 'host2'])
df.head()

,request_method,user_id,cohort_id,ip,network,host
ds,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,97105,1961
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,97105,1961
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,97105,1961
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,97105,1961
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,97105,1961


In [18]:
df.network.nunique()

715

In [19]:
df.host.nunique()

3909

**I'll check for (and remove) outliers in IPs' network and host seperately.**

In [20]:
df.network.value_counts().tail(100)

165227    4
46218     4
24167     3
75131     3
3895      3
         ..
743       1
12252     1
38140     1
67220     1
476       1
Name: network, Length: 100, dtype: int64

In [21]:
df = df.groupby('network').filter(lambda x : len(x)>3)
df.network.value_counts()

97105     329050
6711       22545
13650      13908
76185      11978
72181       8998
           ...  
1228           4
12151          4
189166         4
6642           4
70158          4
Name: network, Length: 617, dtype: int64

In [22]:
df.shape

(673349, 6)

In [23]:
df.host.value_counts().tail(800)

3231      2
107245    2
203140    2
21261     2
10172     2
         ..
213177    1
637       1
23228     1
37210     1
166181    1
Name: host, Length: 800, dtype: int64

In [24]:
df = df.groupby('host').filter(lambda x : len(x)>2)
df.host.value_counts()

1958      268254
1961       60492
117210      8894
21733       4913
208194      4261
           ...  
220191         3
244130         3
155102         3
21966          3
47106          3
Name: host, Length: 2941, dtype: int64

In [25]:
df.shape

(672117, 6)

I might choose to explore hosts with only one-time access at a later point, in which case I would re-add these dropped observations.

In [26]:
# create dataframe out of value counts
ip_counts = pd.DataFrame(df.ip.value_counts()).reset_index()

# rename columns
ip_counts.columns=['ip', 'event_count']

# get the number of ip addresses seen 1, or 2, or 11,998 times. 
ip_counts.groupby(['event_count']).count()

,ip
event_count,
1,34
2,24
3,339
4,285
5,198
...,...
4261,1
4913,1
8894,1


In [27]:
# pass the entropy function an array-like object of counts. 
entropy(ip_counts.event_count)

4.177320753217176

### Hypothesis 1:
    
    